<a href="https://colab.research.google.com/github/johanhoffman/DD2363-VT19/blob/jledeus/jledeus-lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: Direct methods**
**Johan Ledéus**

# **Abstract**

This is labwork for DD2363

#**Mandatory assignment**

###1. Function: QR factorization

####Input: matrix A
####Output: orthogonal matrix Q, upper triangular matrix R, such that A=QR
####Test: R upper triangular, Frobenius norms $$|| Q^TQ-I ||_F, || QR-A ||_F$$

####Solution:
Given matrix $A$ we want to find the orthogonal matrix $Q$ and the upper triangulat matrix $R$.

1. Take the columns of $A$ and make it to an orthonormal set with Gram Schmidt. That will result in the orthogonal matrix $Q$. 

2. Since $Q$ is an orthonormal ($Q^{-1} = Q^T$) we know that $R = Q^T \cdot A$

#### Classical Gram-Schmidt orthogonalization (5.2)

$$v_j = a_j - \sum_{i=1}^{j-1}(a_j,q_i)q_i$$
$$q_j = v_j/||v_j||$$ 


In [0]:
##### Code
#### Dependencies
import numpy as np

# Gram schmidt
def gram_schmidt(A):
  Q = np.zeros(A.shape) # Result matrix with zero
  # Sum part of equation above
  for j in range(len(A)):
    sum_part = np.zeros(np.size(A,1))
    for i in range(j):
      sum_part += np.dot(A[:,j],Q[:,i])*Q[:,i]
    # Assign value and normalize the vector
    Q[:,j] = A[:,j] - sum_part
    Q[:,j] /= np.linalg.norm(Q[:,j])
  return Q

def qr_factorization(A):
  Q = gram_schmidt(A)
  R = np.matmul(np.transpose(Q),A)
  return Q, R

#### For testing we make sure that R is an upper triangular matrix and compare the Frobenius norms $|| Q^TQ-I ||_F, || QR-A ||_F$

$$||A||_F = \sqrt{\sum_{i=1}^m\sum_{j=1}^na_{ij}^2}$$ 

In [124]:
# Upper triangular matrix test
def upper_triangular_test(R):
  for i in range(len(R)):
    for j in range(i):
      if R[i,j] > 0.00000000001: # Small epsilon value
        return False
  return True

def frobenius_norms(M):
  res = 0
  for i in M:
    for j in i:
      res += j**2
  return True if np.sqrt(res) < 0.00000000001 else False

A = np.array([(1,1,0),(1,0,1),(0,1,1)])
Q, R = qr_factorization(A)
assert upper_triangular_test(R) == True
assert frobenius_norms(np.matmul(np.transpose(Q),Q)-np.identity(len(Q))) == True
assert frobenius_norms(np.matmul(Q,R)-A) == True
print("Pass")

Pass


###2. Function: direct solver Ax=b

####Input: matrix $A$, vector $b$
####Output: vector $x=A^{-1}b$
####Test: residual $|| Ax-b ||$, and $|| x-y ||$ where y is a manufactured solution with $b=Ay$

#### Solution
Given that matrix **A** is non-singular, meaning that it's invertible,  we can indirectly compute its inverse. 

$$A = QR$$
$$Ax=b \iff  QRx=b \iff Rx =Q^{-1}b=Q^{T}b$$
The inverse of the orthogonal matrix **Q** is its transpose.
Since **R** is an upper triangular matrix we can use the backward substitition (*Course litterature page 69*) 


In [0]:
def direct_solver(A,b):
  Q,R = qr_factorization(A)
  b = np.transpose(Q).dot(b)
  x = np.zeros(len(b))
  
  # Backward substitution 
  for i in range(len(b)-1, -1, -1):
    x[i] = (b[i]-sum([R[i][j]*x[j] for j in range(i+1,len(b))]))/R[i][i]
  
  return x

In [126]:
# Test code
A = np.array([(1,1,1),(0,2,5),(2,5,-1)])
b = np.array([6,-4,27])
x = direct_solver(A,b)
y = np.linalg.inv(A).dot(b) # Manufactured solution

assert np.linalg.norm(A.dot(x) - b) < 0.0000000001
assert np.linalg.norm(x-y) < 0.0000000001
print("Pass")


Pass


# **Extra assignment**

###3. Function: least squares problem $Ax=b$

####Input: matrix $A$, vector $b$
####Output: vector $x$ 
####Test: residual $|| Ax-b ||$

#### Solution (5.4 in course litterature)
If the system is overdetermined and there exists no precise solution we want to find $x \in \Bbb R^n $ that minimize $r = b-Ax$, so that

$$||b-Ax|| \leq ||b-Ay||, \forall y \in \Bbb R^n $$

Given this we want to calculate the projected vector $Ax$ which is perpendicular to $r=b-Ax$. Since it's perpendicular to $range(A) \implies A^Tr = 0 $.

$$A^TAx = A^Tb-A^Tr = A^Tb \implies x=(A^TA)^{-1}A^Tb $$

In [0]:
def least_squares(A,b):
  b = np.transpose(A).dot(b) # A^Tb
  A = np.matmul(np.transpose(A),A) # A^TA
  return direct_solver(A,b)

In [128]:
# Test code
A = np.array([(1,-1),(1,1),(2,1)])
b = np.array([2,4,8])
x = least_squares(A,b)
y = np.linalg.lstsq(A,b, rcond=-1)[0] # Manufactured solution

# compare with library solution
assert np.abs(np.linalg.norm(A.dot(x) - b) - np.linalg.norm(A.dot(y) - b)) < 0.00000001
# Extra test, the dot product of r and Ax should be 0
assert (b-A.dot(x)).dot(A.dot(x)) < 0.00000000001

print("Pass")

Pass


###4. Function: QR eigenvalue algorithm

####Input: real symmetric matrix $A$
####Output: real eigenvalues $\lambda_i$ and real eigenvectors $v_i$ of $A$
####Test: $det(A - \lambda_i I), || Av_i - \lambda_i v_i ||$  



#### Solution (6.2 in course litterature)
With the QR eigenvalue algorithm the input matrix $A$ will converge to an upper triangular matrix. One property of an upper triangular matrix is that the diagonal values are in fact the eigenvalues.

We can find the eigenvectors in the columns of: $\prod_iQ_i$

[External resources](https://people.kth.se/~eliasj/qrmethod.pdf)

In [0]:
def qr_eigenvalue_algorithm(A):
  eigenvectors = np.identity(len(A[0]))
  while not upper_triangular_teset(A):
    Q,R = qr_factorization(A)
    A = np.matmul(R,Q)
    eigenvectors = np.matmul(eigenvectors, Q)
  return A.diagonal(), np.transpose(eigenvectors)  

In [130]:
# Test code

A = np.array([(2,1,-3),(1,0,4),(-3,4,-1)])
eigenvalues, eigenvectors = qr_eigenvalue_algorithm(A)

for eigen in eigenvalues:
  assert np.linalg.det(A - eigen*np.identity(len(A))) < 0.00000001
  
for i in range(len(eigenvectors)):
  assert np.linalg.norm(np.abs(A.dot(eigenvectors[i]) - eigenvalues[i]*eigenvectors[i])) < 0.00000001
print("pass")

pass


# Discussion

This lab was more challenging than the previous one. But at the same time it was fun to repeat some old linear algebra. The hardest part was to construct the eigenvectors in assignment 4 since it wasn't that intuitive.